<a href="https://colab.research.google.com/github/Vyoma-garg/Deep_Learning/blob/main/Finetuning_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import torch
import torch.nn as nn  #all the NN network, CNN, Modules, loss functions, activation fns
import torch.optim as optim  #SGD, ADAM
import torch.nn.functional as F  #activation fn, 
from torch.utils.data import DataLoader  #easier data managemnet, creates mini batches of the data
import torchvision.datasets as datasets   #standard datasets MNISt ,etc
import torchvision.transforms as transforms  # transformations 
import torchvision

Setting device

In [46]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

Hyperparameters

In [47]:
in_channels=3
num_classes=10
learning_rate=0.001
batch_size=1024
num_epochs=5

Creating Identity class to do nothing where needed

In [48]:
class Identity(nn.Module): #inherit from nn.Module class
    def __init__(self):  
        super(Identity,self).__init__()   

    def forward(self, x):    
        return x


# **Loading the pretrained model (pretrained=True)**

In [30]:
model=torchvision.models.vgg16(pretrained=True)
print(model)

# If you want to do finetuning --> TRANSFER LEARNING then set requires_grad = False
# Remove these two lines if you want to train entire model, and only want to load the pretrain weights.

for param in model.parameters():
    param.requires_grad = False #DOES NOT CHANGE THE WEIGHTS 

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Modifying the model

In [31]:
#model.avgpool= Identity()
#model.classifier= nn.Linear(512,10)
#model.to(device)
#print(model)

#same 
#model.avgpool=Identity()
#model.classifier[0]= nn.Linear(512,10)
#for i in range(1,7):
# model.classifier[i]= Identity

#diff
model.avgpool=Identity()
model.classifier= nn.Sequential(nn.Linear(512,100),
                                nn.ReLU(),
                               nn.Linear(100,10))
model.to(device)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Loading dataset

In [49]:
train_dataset = datasets.CIFAR10(root = '/content', train = True , transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.CIFAR10(root = '/content', train = False , transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

Files already downloaded and verified
Files already downloaded and verified


Loss and Optimizer

In [50]:
criterion= nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

Train Network

In [33]:
for epoch in range(num_epochs):
  losses = []

  for batch_idx,(data,targets) in enumerate(train_loader):  #enumerate helps in getting the batch index of the data
     
     #Get data to cuda
      data = data.to(device)
      targets = targets.to(device)

      #print(data.shape)  #([64,1,28,28])  num of examples, number of channels(black white img), height width of each image
                        
      #get correct shape
      #data=data.reshape(data.shape[0],-1)  #flatten to single dim: ([64,784])
      #print(data.shape) 

      #forward
      scores=model(data)
      loss=criterion(scores, targets)
      losses.append(loss.item())

      #backward
      optimizer.zero_grad() #
      loss.backward()

      #gradient descent or adam step
      optimizer.step() #update weights based on the gradients
  print(f"Loss at epoch {epoch} is { sum(losses)/len(losses) :.5f}")  


Loss at epoch 0 is 1.59687
Loss at epoch 1 is 1.21371
Loss at epoch 2 is 1.14643
Loss at epoch 3 is 1.11209
Loss at epoch 4 is 1.09173


Check accuracy

In [35]:
def check_accuracy(loader,model):
  if loader.dataset.train:
        print('Checking Accuracy on Training Data')
  else:
        print('Checking Accuracy on Test Data')
  num_correct=0
  num_samples=0
  model.eval()

  with torch.no_grad():  #while checking accuracy no need to calculate the gradients again
    for x,y in loader:
      x=x.to(device=device)
      y=y.to(device=device)
      #x=x.reshape(x.shape[0],-1)

      scores=model(x)  #64x10 shape of the scores, which is max of those 10 digits
      values,predictions =scores.max(1)  #interested in index of the max value
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)  #first dimension that is 64 x 10==64
    print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} \n')

  model.train()
  

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)



Checking Accuracy on Training Data
Got 31373/ 50000 with accuracy 62.75 

Checking Accuracy on Test Data
Got 6119/ 10000 with accuracy 61.19 



# **WHEN PRETRAINED IS FALSE**

In [51]:
model=torchvision.models.vgg16(pretrained=False)
print(model)

# If you want to do finetuning --> TRANSFER LEARNING then set requires_grad = False
# Remove these two lines if you want to train entire model, and only want to load the pretrain weights.

#for param in model.parameters():
#    param.requires_grad = False #DOES NOT CHANGE THE WEIGHTS 

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Modifying the model

In [52]:
#model.avgpool= Identity()
#model.classifier= nn.Linear(512,10)
#model.to(device)
#print(model)

#same 
#model.avgpool=Identity()
#model.classifier[0]= nn.Linear(512,10)
#for i in range(1,7):
# model.classifier[i]= Identity

#diff
model.avgpool=Identity()
model.classifier= nn.Sequential(nn.Linear(512,100),
                                nn.ReLU(),
                               nn.Linear(100,10))
model.to(device)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Train Network

In [53]:
for epoch in range(num_epochs):
  losses = []

  for batch_idx,(data,targets) in enumerate(train_loader):  #enumerate helps in getting the batch index of the data
     
     #Get data to cuda
      data = data.to(device)
      targets = targets.to(device)

      #print(data.shape)  #([64,1,28,28])  num of examples, number of channels(black white img), height width of each image
                        
      #get correct shape
      #data=data.reshape(data.shape[0],-1)  #flatten to single dim: ([64,784])
      #print(data.shape) 

      #forward
      scores=model(data)
      loss=criterion(scores, targets)
      losses.append(loss.item())

      #backward
      optimizer.zero_grad() #
      loss.backward()

      #gradient descent or adam step
      optimizer.step() #update weights based on the gradients
  print(f"Loss at epoch {epoch} is { sum(losses)/len(losses) :.5f}")  


Loss at epoch 0 is 2.30402
Loss at epoch 1 is 2.30402
Loss at epoch 2 is 2.30403
Loss at epoch 3 is 2.30402
Loss at epoch 4 is 2.30403


Check accuracy

In [54]:
def check_accuracy(loader,model):
  if loader.dataset.train:
        print('Checking Accuracy on Training Data')
  else:
        print('Checking Accuracy on Test Data')
  num_correct=0
  num_samples=0
  model.eval()

  with torch.no_grad():  #while checking accuracy no need to calculate the gradients again
    for x,y in loader:
      x=x.to(device=device)
      y=y.to(device=device)
      #x=x.reshape(x.shape[0],-1)

      scores=model(x)  #64x10 shape of the scores, which is max of those 10 digits
      values,predictions =scores.max(1)  #interested in index of the max value
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)  #first dimension that is 64 x 10==64
    print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} \n')

  model.train()
  

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)



Checking Accuracy on Training Data
Got 5000/ 50000 with accuracy 10.00 

Checking Accuracy on Test Data
Got 1000/ 10000 with accuracy 10.00 

